In [2]:
from enum import Enum
from pydantic import BaseModel, Field, AfterValidator, WithJsonSchema, PlainSerializer
from typing_extensions import Annotated


class ZepDataType(Enum):
    # Define the enum values here, assuming placeholder values
    TYPE1 = "type1"
    TYPE2 = "type2"
    
class ZepFloat(BaseModel):
    description: str
    name: str
    type: float
    regex: str

In [3]:
TruncatedFloat = Annotated[
    float,
    AfterValidator(lambda x: round(x, 1)),
    PlainSerializer(lambda x: f'{x:.1e}', return_type=str),
    WithJsonSchema(ZepFloat.model_json_schema(), mode='serialization'),
]

In [4]:
class TestModel(BaseModel):
    a: TruncatedFloat = Field(..., title="A", description="This is a description")
    b: ZepFloat = Field(..., title="B", description="This is a descriptionB")

In [5]:
for field in TestModel.model_fields.items():
    print(field[1].annotation)

<class 'float'>
<class '__main__.ZepFloat'>


In [22]:
import typing
from typing import Optional


class ZepBaseFloat(BaseModel):
    description: typing.Optional[str] = None
    type: ZepDataType = "ZepFloat"


ZepFloat = Annotated[
    float,
    # AfterValidator(lambda x: round(x, 1)),
    PlainSerializer(lambda x: f'{x:.1e}', return_type=str),
    WithJsonSchema(ZepBaseFloat.model_json_schema(), mode='serialization'),
]

class ShoeInfoModel(BaseModel):
    shoe_size: ZepFloat = Field(default=None, description="This is a field from the base model")
    
for field in ShoeInfoModel.model_fields.items():
    print(field[1].annotation)
    
t = ShoeInfoModel(shoe_size=1.234)
t

<class 'float'>


ShoeInfoModel(shoe_size=1.234)

In [23]:
t.model_dump_json()
t

'{"shoe_size":"1.2e+00"}'